# Classify snow-covered area (SCA) in Sentinel-2, Landsat 8/9, and PlanetScope imagery: full pipelines

Rainey Aberle

Department of Geosciences, Boise State University

2022

### Requirements:
- Area of Interest (AOI) shapefile: where snow will be classified in all available images. 
- Google Earth Engine (GEE) account: used to pull DEM over the AOI. Sign up for a free account [here](https://earthengine.google.com/new_signup/). 
- Digital elevation model (DEM) (_optional_): used to extract elevations over the AOI and for each snowline. If no DEM is provided, the ASTER Global DEM will be loaded through GEE. 

### Outline:
__0. Setup__ paths in directory, file locations, authenticate GEE - _modify this section!_

__1. Sentinel-2 Top of Atmosphere (TOA) imagery:__ full pipeline

__2. Sentinel-2 Surface Reflectance (SR) imagery:__ full pipeline

__3. Landsat 8/9 Surface Reflectance (SR) imagery:__ full pipeline

__4. PlanetScope Surface Reflectance (SR) imagery:__ full pipeline

-------


### 0. Setup

#### Define paths in directory and desired settings. 
Modify lines located within the following:

`#### MODIFY HERE ####`  

`#####################`

In [ ]:
##### MODIFY HERE #####

# -----Paths in directory
site_name = 'Gulkana'
# path to snow-cover-mapping/ - Make sure you include a "/" at the end
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/'
# path to folder containing AOI files
AOI_path = '/Users/raineyaberle/Google Drive/My Drive/Research/PhD/snow_cover_mapping/snow_cover_mapping_application/study-sites/' + site_name + '/AOIs/'
# AOI file name
AOI_fn = 'Gulkana_USGS_glacier_outline_2021.shp' 
# path to folder containing DEM raster file
# Note: set DEM_fn=None if you want to use the ArcticDEM or ASTER GDEM via Google Earth Engine
DEM_path = AOI_path + '../DEMs/'
# DEM file name
DEM_fn = 'Gulkana_ArcticDEM_clip.tif'
# path for output images
out_path = AOI_path + '../imagery/'
# path to PlanetScope images
# Note: set PS_im_path=None if not using PlanetScope
PS_im_path = out_path + 'PlanetScope/raw_images/'
# path for output figures
figures_out_path = AOI_path + '../figures/'

# -----Define image search filters
date_start = '2021-05-01'
date_end = '2021-12-01'
month_start = 5
month_end = 10
cloud_cover_max = 70

# -----Determine whether to mask clouds using the respective cloud masking data products
# NOTE: Cloud mask products anecdotally are less accurate over glacierized/snow-covered surfaces. 
# If the cloud masks are consistently masking large regions or your study site, I suggest setting mask_clouds = False
mask_clouds = True

# -----Determine image download, clipping & plotting settings
# Note: if im_download = False, but images over the AOI exceed GEE limit,
# images must be downloaded regardless.
im_download = False  # = True to download all satellite images by default
plot_results = True # = True to plot figures of results for each image where applicable
skip_clipped = False # = True to skip images where bands appear "clipped", i.e. max(blue) < 0.8
crop_to_AOI = True # = True to crop images to AOI before calculating SCA
save_outputs = True # = True to save SCAs and snowlines to file
save_figures = True # = True to save output figures to file

#######################

# -----Import packages
import xarray as xr
import os
import numpy as np
import glob
from matplotlib import pyplot as plt, dates
import matplotlib
import rasterio as rio
import geopandas as gpd
import pandas as pd
import sys
import ee
import geedim as gd
import json
from tqdm.auto import tqdm
from joblib import dump, load
from shapely.geometry import MultiPolygon, Polygon

# -----Set paths for output files
S2_TOA_im_path = out_path + 'Sentinel-2_TOA/'
S2_SR_im_path = out_path + 'Sentinel-2_SR/'
L_im_path = out_path + 'Landsat/'
PS_im_masked_path = out_path + 'PlanetScope/masked/'
PS_im_mosaics_path = out_path + 'PlanetScope/mosaics/'
im_classified_path = out_path + 'classified/'
snowlines_path = out_path + 'snowlines/'

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
import pipeline_utils as f

# -----Load dataset dictionary
dataset_dict = json.load(open(base_path + 'inputs-outputs/datasets_characteristics.json'))

#### Authenticate and initialize Google Earth Engine (GEE). 

__Note:__ The first time you run the following cell, you will be asked to authenticate your GEE account for use in this notebook. This will send you to an external web page, where you will walk through the GEE authentication workflow and copy an authentication code back into the space below this cell when prompted. 

In [ ]:
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except: 
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

#### Load AOI and DEM

In [ ]:
# -----Load AOI as gpd.GeoDataFrame
AOI = gpd.read_file(AOI_path + AOI_fn)
# reproject the AOI to WGS to solve for the optimal UTM zone
AOI_WGS = AOI.to_crs('EPSG:4326')
AOI_WGS_centroid = [AOI_WGS.geometry[0].centroid.xy[0][0],
                    AOI_WGS.geometry[0].centroid.xy[1][0]]
# grab the optimal UTM zone EPSG code
epsg_UTM = f.convert_wgs_to_utm(AOI_WGS_centroid[0], AOI_WGS_centroid[1])
print('Optimal UTM CRS = EPSG:' + str(epsg_UTM))
# reproject AOI to the optimal UTM zone
AOI_UTM = AOI.to_crs('EPSG:'+epsg_UTM)

# -----Load DEM as Xarray DataSet
if DEM_fn is None:
    # query GEE for DEM
    DEM = f.query_gee_for_dem(AOI_UTM, base_path, site_name, DEM_path)
else:
    # load DEM as xarray DataSet
    DEM = xr.open_dataset(DEM_path + DEM_fn)
    DEM = DEM.rename({'band_data': 'elevation'})
    # reproject the DEM to the optimal UTM zone
    DEM = DEM.rio.reproject('EPSG:'+str(epsg_UTM))
    DEM = DEM.rio.write_crs('EPSG:'+str(epsg_UTM))
# remove unnecessary data (possible extra bands from ArcticDEM or other DEM)
if len(np.shape(DEM.elevation.data))>2:
    DEM['elevation'] = DEM.elevation[0]
    DEM = xr.where(DEM < -100, np.nan, DEM)
    DEM = DEM.rio.write_crs('EPSG:'+str(epsg_UTM))

# -----Plot
fig, ax = plt.subplots(1, 1, figsize=(6,6))
dem_im = ax.imshow(DEM.elevation.data, cmap='terrain', 
          extent=(np.min(DEM.x.data)/1e3, np.max(DEM.x.data)/1e3, np.min(DEM.y.data)/1e3, np.max(DEM.y.data)/1e3))
if type(AOI_UTM.geometry[0])==Polygon:
    ax.plot([x/1e3 for x in AOI_UTM.geometry[0].exterior.coords.xy[0]],
            [y/1e3 for y in AOI_UTM.geometry[0].exterior.coords.xy[1]], '-k')
elif type(AOI_UTM.geometry[0])==MultiPolygon:
    [ax.plot([x/1e3 for x in geom.exterior.coords.xy[0]],
            [y/1e3 for y in geom.exterior.coords.xy[1]], '-k') for geom in AOI_UTM.geometry[0].geoms]
ax.grid()
ax.set_xlabel('Easting [km]')
ax.set_ylabel('Northing [km]')
fig.colorbar(dem_im, ax=ax, shrink=0.5, label='Elevation [m]')
plt.show()

## 1. Sentinel-2 TOA imagery

In [ ]:
# -----Query GEE for imagery (and download to S2_TOA_im_path if necessary)
dataset = 'Sentinel-2_TOA'
im_list = f.query_gee_for_imagery(dataset_dict, dataset, AOI_UTM, date_start, date_end, month_start, 
                                  month_end, cloud_cover_max, mask_clouds, S2_TOA_im_path, im_download)

In [ ]:
# -----Load trained classifier and feature columns
clf_fn = base_path+'inputs-outputs/Sentinel-2_TOA_classifier_all_sites.joblib'
clf = load(clf_fn)
feature_cols_fn = base_path+'inputs-outputs/Sentinel-2_TOA_feature_columns.json'
feature_cols = json.load(open(feature_cols_fn))

# -----Loop through images
if type(im_list)==str: # check that images were found
    print('No images found to classify, quiting...')
else:
    
    for i in tqdm(range(0, len(im_list))):
        
        # -----Subset image using loop index
        im_xr = im_list[i]
        im_date = str(im_xr.time.data[0])[0:19]
        print(im_date)
        
        # -----Adjust image for image scalar and no data values
        # replace no data values with NaN and account for image scalar
        crs = im_xr.rio.crs.to_epsg()
        if np.nanmean(im_xr['B2'])>1e3:
            im_xr = xr.where(im_xr==dataset_dict[dataset]['no_data_value'], np.nan, 
                             im_xr / dataset_dict[dataset]['image_scalar'])
        else:
            im_xr = xr.where(im_xr==dataset_dict[dataset]['no_data_value'], np.nan, im_xr)
        # add NDSI band
        im_xr['NDSI'] = ((im_xr[dataset_dict[dataset]['NDSI_bands'][0]] - im_xr[dataset_dict[dataset]['NDSI_bands'][1]]) 
                             / (im_xr[dataset_dict[dataset]['NDSI_bands'][0]] + im_xr[dataset_dict[dataset]['NDSI_bands'][1]]))
        im_xr.rio.write_crs('EPSG:'+str(crs), inplace=True)
                
        # -----Classify image
        # check if classified image already exists in file
        im_classified_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_classified.nc'
        if os.path.exists(im_classified_path + im_classified_fn):
            print('Classified image already exists in file, continuing...')
            im_classified = xr.open_dataset(im_classified_path + im_classified_fn)
            # remove no data values
            im_classified = xr.where(im_classified==-9999, np.nan, im_classified)
        else:  
            # classify image
            im_classified = f.classify_image(im_xr, clf, feature_cols, crop_to_AOI, AOI_UTM, DEM,
                                             dataset_dict, dataset, im_classified_fn, im_classified_path)
            if type(im_classified)==str: # skip if error in classification
                continue
        
        # -----Delineate snowline(s)
        # check if snowline already exists in file
        snowline_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_snowline.csv'
        if os.path.exists(snowlines_path + snowline_fn):
            print('Snowline already exists in file, continuing...')
            print(' ')
            continue # no need to load snowline if it already exists
        else:
            plot_results = True
            # create directory for figures if it doesn't already exist
            if (not os.path.exists(figures_out_path)) & plot_results:
                os.mkdir(figures_out_path)
                print('Created directory for output figures: '+figures_out_path)
            snowline_df = f.delineate_image_snowline(im_xr, im_classified, site_name, AOI_UTM, dataset_dict, dataset, 
                                                     im_date, snowline_fn, snowlines_path, figures_out_path, plot_results)
            # plt.show()
            print('Accumulation Area Ratio =  ' + str(snowline_df['AAR'][0]))
        print(' ')

## 2. Sentinel-2 SR imagery

In [ ]:
# -----Query GEE for imagery and download to S2_SR_im_path if necessary
dataset = 'Sentinel-2_SR'
im_list = f.query_gee_for_imagery(dataset_dict, dataset, AOI_UTM, date_start, date_end, month_start, 
                                  month_end, cloud_cover_max, mask_clouds, S2_SR_im_path, im_download)

In [ ]:
from shapely.geometry import MultiPolygon, Polygon, MultiLineString, LineString, Point, shape
from scipy.ndimage import binary_fill_holes, binary_dilation
from skimage.measure import find_contours
from scipy.interpolate import interp1d
from scipy.stats import iqr

def delineate_snowline(im_xr, im_classified, site_name, aoi, dataset_dict, dataset, im_date, snowline_fn,
                       out_path, figures_out_path, plot_results, verbose=False):
    """
    Delineate the seasonal snowline in classified images. Snowlines will likely not be detected in images with nearly all or no snow.

    Parameters
    ----------
    im_xr: xarray.Dataset
        input image used for plotting
    im_classified: xarray.Dataset
        classified image used to delineate snowlines
    site_name: str
        name of study site used for output file names
    aoi:  geopandas.geodataframe.GeoDataFrame
        area of interest used to crop classified images
    dataset_dict: dict
        dictionary of dataset-specific parameters
    dataset: str
        name of dataset ('Landsat', 'Sentinel2', 'PlanetScope')
    im_date: str
        image capture datetime ('YYYYMMDDTHHmmss')
    snowline_fn: str
        file name of snowline to be saved in out_path
    out_path: str
        path in directory for output snowlines
    figures_out_path: str
        path in directory for figures
    plot_results: bool
        whether to plot RGB image, classified image, and resulting snowline and save figure to file
    verbose: bool
        whether to print details during the process

    Returns
    ----------
    snowline_gdf: geopandas.GeoDataFrame
        resulting study site name, image datetime, snowline coordinates, snowline elevations, and median snowline elevation
    """

    # -----Make directory for snowlines (if it does not already exist)
    if not os.path.exists(out_path):
        os.mkdir(out_path)
        print("Made directory for snowlines:" + out_path)
        
    # -----Make directory for figures (if it does not already exist)
    if (not os.path.exists(figures_out_path)) & plot_results:
        os.mkdir(figures_out_path)
        print('Made directory for output figures: ' + figures_out_path)

    # -----Subset dataset_dict to dataset
    ds_dict = dataset_dict[dataset]

    # -----Remove time dimension
    im_xr = im_xr.isel(time=0)
    im_classified = im_classified.isel(time=0)

    # -----Create no data mask
    no_data_mask = xr.where(np.isnan(im_classified), 1, 0).classified.data
    # dilate by two image pixels
    dilated_mask = binary_dilation(no_data_mask, iterations=5)
    no_data_mask = np.logical_not(dilated_mask)
    # add no_data_mask variable classified image
    im_classified = im_classified.assign(no_data_mask=(["y", "x"], no_data_mask))

    # -----Determine snow covered elevations
    all_elev = np.ravel(im_classified.elevation.data)
    all_elev = all_elev[~np.isnan(all_elev)]  # remove NaNs
    snow_est_elev = np.ravel(im_classified.where((im_classified.classified <= 2))
                             .where(im_classified.classified != -9999).elevation.data)
    snow_est_elev = snow_est_elev[~np.isnan(snow_est_elev)]  # remove NaNs

    # -----Create elevation histograms
    # determine bins to use in histograms
    elev_min = np.fix(np.nanmin(np.ravel(im_classified.elevation.data)) / 10) * 10
    elev_max = np.round(np.nanmax(np.ravel(im_classified.elevation.data)) / 10) * 10
    bin_edges = np.linspace(elev_min, elev_max, num=int((elev_max - elev_min) / 10 + 1))
    bin_centers = (bin_edges[1:] + bin_edges[0:-1]) / 2
    # calculate elevation histograms
    hist_elev = np.histogram(all_elev, bins=bin_edges)[0]
    hist_snow_est_elev = np.histogram(snow_est_elev, bins=bin_edges)[0]
    hist_snow_est_elev_norm = hist_snow_est_elev / hist_elev

    # -----Make all pixels at elevation bins with >75% snow coverage = snow
    # determine elevation with > 75% snow coverage
    if np.any(hist_snow_est_elev_norm > 0.75):
        elev_75_snow = bin_centers[np.argmax(hist_snow_est_elev_norm > 0.75)]
        # make a copy of im_classified for adjusting
        im_classified_adj = im_classified.copy()
        # Fill gaps in elevation using linear interpolation along the spatial dimensions
        im_classified_adj['elevation'] = im_classified['elevation'].interpolate_na(dim='x', method='linear')
        # set all pixels above the elev_75_snow to snow (1)
        im_classified_adj['classified'] = xr.where(im_classified_adj['elevation'] > elev_75_snow, 1,
                                                   im_classified_adj['classified'])
        # create a binary mask for everything above the first instance of 75% snow-covered
        elevation_threshold_mask = xr.where(im_classified.elevation > elev_75_snow, 1, 0) 
        
    else:
        im_classified_adj = im_classified

    # -----Delineate snow lines
    # create binary snow matrix
    im_binary = xr.where(im_classified_adj > 2, 1, 0).classified.data
    # fill holes in binary image (0s within 1s = 1)
    im_binary_no_holes = binary_fill_holes(im_binary)
    # find contours at a constant value of 0.5 (between 0 and 1)
    contours = find_contours(im_binary_no_holes, 0.5)    
    # convert contour points to image coordinates
    contours_coords = []
    for contour in contours:
        # convert image pixel coordinates to real coordinates
        fx = interp1d(range(0, len(im_classified_adj.x.data)), im_classified_adj.x.data)
        fy = interp1d(range(0, len(im_classified_adj.y.data)), im_classified_adj.y.data)
        coords = (fx(contour[:, 1]), fy(contour[:, 0]))
        # zip points together
        xy = list(zip([x for x in coords[0]],
                      [y for y in coords[1]]))
        contours_coords.append(xy)
            
    # convert list of coordinates to list of LineStrings
    # do not include points in the no data mask or points above the elevation threshold
    contour_lines = []
    for contour_coords in contours_coords:
        points_real = [Point(x,y) for x,y in contour_coords
                       if (im_classified.sel(x=x, y=y, method='nearest').no_data_mask.data.item()==True)
                       and (elevation_threshold_mask.sel(x=x, y=y, method='nearest').data.item()==0)]
        
        if len(points_real)>2: # need at least 3 points for a LineString
            contour_line = LineString([[point.x, point.y] for point in points_real])
            contour_lines.append(contour_line)
    
    # proceed if lines were found after filtering
    if len(contour_lines) > 0:
        
        # -----Use the longest line as the snowline
        lengths = [line.length for line in contour_lines]
        max_length_index = max(range(len(contour_lines)), key=lambda i: lengths[i])
        snowline = contour_lines[max_length_index]
        
        # -----Interpolate elevations at snow line coordinates
        # compile all line coordinates into arrays of x- and y-coordinates
        xpts = np.ravel([x for x in snowline.coords.xy[0]])
        ypts = np.ravel([y for y in snowline.coords.xy[1]])
        # interpolate elevation at snow line points
        snowline_elevs = [im_classified.sel(x=x, y=y, method='nearest').elevation.data.item()
                          for x, y in list(zip(xpts, ypts))]
        
    else:
        
        snowline = []
        snowline_elevs = np.nan
        
    # -----If AOI is ~covered in snow, set snowline elevation to the minimum elevation in the AOI
    if np.all(np.isnan(snowline_elevs)) and (np.nanmedian(hist_snow_est_elev_norm) > 0.5):
        snowline_elevs = np.nanmin(np.ravel(im_classified.elevation.data))
        
    # -----Calculate snow-covered area (SCA) and accumulation area ratio (AAR)
    # pixel resolution
    dx = im_classified.x.data[1] - im_classified.x.data[0]
    # snow-covered area
    sca = len(np.ravel(im_classified.classified.data[im_classified.classified.data <= 2])) * (
            dx ** 2)  # number of snow-covered pixels * pixel resolution [m^2]
    # accumulation area ratio
    total_area = len(np.ravel(im_classified.classified.data[~np.isnan(im_classified.classified.data)])) * (
            dx ** 2)  # number of pixels * pixel resolution [m^2]
    aar = sca / total_area
    
    # -----Compile results in dataframe
    # calculate median snow line elevation
    median_snowline_elev = np.nanmedian(snowline_elevs)
    # compile results in df
    if type(snowline)==LineString:
        snowlines_coords_x = [ snowline.coords.xy[0] ]
        snowlines_coords_y = [ snowline.coords.xy[1] ]
    else:
        snowlines_coords_x = [[]]
        snowlines_coords_y = [[]]
    snowline_df = pd.DataFrame({'site_name': [site_name],
                                'datetime': [im_date],
                                'snowlines_coords_X': snowlines_coords_x,
                                'snowlines_coords_Y': snowlines_coords_y,
                                'CRS': ['EPSG:' + str(im_xr.rio.crs.to_epsg())],
                                'snowline_elevs_m': [snowline_elevs],
                                'snowline_elevs_median_m': [median_snowline_elev],
                                'SCA_m2': [sca],
                                'AAR': [aar],
                                'dataset': [dataset],
                                'geometry': [snowline]
                                })
    
     # -----Save snowline df to file
    # reduce memory storage of dataframe
    # snowline_df = f.reduce_memory_usage(snowline_df, verbose=False)
    # save using user-specified file extension
    # if 'pkl' in snowline_fn:
    #     snowline_df.to_pickle(out_path + snowline_fn)
    #     if verbose:
    #         print('Snowline saved to file: ' + out_path + snowline_fn)
    # elif 'csv' in snowline_fn:
    #     snowline_df.to_csv(out_path + snowline_fn, index=False)
    #     if verbose:
    #         print('Snowline saved to file: ' + out_path + snowline_fn)
    # else:
    #     print('Please specify snowline_fn with extension .pkl or .csv. Exiting...')
    #     return 'N/A'
    
    # -----Plot results
    if plot_results:
        fig, ax = plt.subplots(2, 2, figsize=(12, 8), gridspec_kw={'height_ratios': [3, 1]})
        ax = ax.flatten()
        # define x and y limits
        xmin, xmax = aoi.geometry[0].buffer(100).bounds[0] / 1e3, aoi.geometry[0].buffer(100).bounds[2] / 1e3
        ymin, ymax = aoi.geometry[0].buffer(100).bounds[1] / 1e3, aoi.geometry[0].buffer(100).bounds[3] / 1e3
        # define colors for plotting
        colors = list(dataset_dict['classified_image']['class_colors'].values())
        cmp = matplotlib.colors.ListedColormap(colors)
        # RGB image
        ax[0].imshow(np.dstack([im_xr[ds_dict['RGB_bands'][0]].data,
                                im_xr[ds_dict['RGB_bands'][1]].data,
                                im_xr[ds_dict['RGB_bands'][2]].data]),
                     extent=(np.min(im_xr.x.data) / 1e3, np.max(im_xr.x.data) / 1e3, np.min(im_xr.y.data) / 1e3,
                             np.max(im_xr.y.data) / 1e3))
        ax[0].set_xlabel('Easting [km]')
        ax[0].set_ylabel('Northing [km]')
        # classified image
        ax[1].imshow(im_classified['classified'].data, cmap=cmp, clim=(1, 5),
                     extent=(np.min(im_classified.x.data) / 1e3, np.max(im_classified.x.data) / 1e3,
                             np.min(im_classified.y.data) / 1e3, np.max(im_classified.y.data) / 1e3))
        # snowline coordinates
        if type(snowline)==LineString:
            ax[0].plot(np.divide(snowline.coords.xy[0], 1e3), np.divide(snowline.coords.xy[1], 1e3),
                       '.', color='#f768a1', markersize=2)
            ax[1].plot(np.divide(snowline.coords.xy[0], 1e3), np.divide(snowline.coords.xy[1], 1e3),
                       '.', color='#f768a1', markersize=2)
        # plot dummy points for legend
        ax[1].scatter(0, 0, color=colors[0], s=50, label='Snow')
        ax[1].scatter(0, 0, color=colors[1], s=50, label='Shadowed snow')
        ax[1].scatter(0, 0, color=colors[2], s=50, label='Ice')
        ax[1].scatter(0, 0, color=colors[3], s=50, label='Rock')
        ax[1].scatter(0, 0, color=colors[4], s=50, label='Water')
        if type(snowline)==LineString:
            ax[0].scatter(0, 0, color='#f768a1', s=25, label='Snowline estimate')        
            ax[1].scatter(0, 0, color='#f768a1', s=25, label='Snowline estimate')
        ax[1].set_xlabel('Easting [km]')
        # AOI
        label='AOI'
        if type(aoi.geometry[0].boundary) == MultiLineString:
            for ii, geom in enumerate(aoi.geometry[0].boundary.geoms):
                if ii >0 :
                    label = '_nolegend_'
                ax[0].plot(np.divide(geom.coords.xy[0], 1e3),
                           np.divide(geom.coords.xy[1], 1e3), '-k', linewidth=1, label=label)
                ax[1].plot(np.divide(geom.coords.xy[0], 1e3),
                           np.divide(geom.coords.xy[1], 1e3), '-k', linewidth=1, label=label)
        elif type(aoi.geometry[0].boundary) == LineString:
            ax[0].plot(np.divide(aoi.geometry[0].boundary.coords.xy[0], 1e3),
                       np.divide(aoi.geometry[0].boundary.coords.xy[1], 1e3), '-k', linewidth=1, label=label)
            ax[1].plot(np.divide(aoi.geometry[0].boundary.coords.xy[0], 1e3),
                       np.divide(aoi.geometry[0].boundary.coords.xy[1], 1e3), '-k', linewidth=1, label=label)
                    
        # reset x and y limits
        ax[0].set_xlim(xmin, xmax)
        ax[0].set_ylim(ymin, ymax)
        ax[1].set_xlim(xmin, xmax)
        ax[1].set_ylim(ymin, ymax)
        # image bands histogram
        ax[2].hist(im_xr[ds_dict['RGB_bands'][0]].data.flatten(), color='blue', histtype='step', linewidth=2,
                   bins=100, label="blue")
        ax[2].hist(im_xr[ds_dict['RGB_bands'][1]].data.flatten(), color='green', histtype='step', linewidth=2,
                   bins=100, label="green")
        ax[2].hist(im_xr[ds_dict['RGB_bands'][2]].data.flatten(), color='red', histtype='step', linewidth=2,
                   bins=100, label="red")
        ax[2].set_xlabel("Surface reflectance")
        ax[2].set_ylabel("Pixel counts")
        ax[2].grid()
        # normalized snow elevations histogram
        ax[3].bar(bin_centers, hist_snow_est_elev_norm, width=(bin_centers[1] - bin_centers[0]), color=colors[0],
                  align='center')
        ax[3].plot([median_snowline_elev, median_snowline_elev], [0, 1], '-', color='#f768a1', 
                   linewidth=3, label='Median snowline elevation')
        ax[3].set_xlabel("Elevation [m]")
        ax[3].set_ylabel("Fraction snow-covered")
        ax[3].grid()
        ax[3].set_xlim(elev_min - 10, elev_max + 10)
        ax[3].set_ylim(0, 1)
        # determine figure title and file name
        title = im_date.replace('-', '').replace(':', '') + '_' + site_name + '_' + dataset + '_snow-cover'
        # add legends
        ax[0].legend(loc='best')
        ax[1].legend(loc='best')
        ax[2].legend(loc='upper right')
        ax[3].legend(loc='lower right')
        fig.suptitle(title)
        fig.tight_layout()
        # save figure
        fig_fn = figures_out_path + title + '.png'
        fig.savefig(fig_fn, dpi=300, facecolor='white', edgecolor='none')
        if verbose:
            print('Figure saved to file:' + fig_fn)
            
    return snowline_df


In [ ]:
# -----Load trained classifier and feature columns
clf_fn = base_path+'inputs-outputs/Sentinel-2_SR_classifier_all_sites.joblib'
clf = load(clf_fn)
feature_cols_fn = base_path+'inputs-outputs/Sentinel-2_SR_feature_columns.json'
feature_cols = json.load(open(feature_cols_fn))

# -----Loop through images
if type(im_list)==str: # check that images were found
    print('No images found to classify, quiting...')
else:
    
    for i in tqdm(range(0, len(im_list))):
        
        # -----Subset image using loop index
        im_xr = im_list[i]
        im_date = str(im_xr.time.data[0])[0:19]
        print(im_date)
        
        # -----Adjust image for image scalar and no data values
        # replace no data values with NaN and account for image scalar
        crs = im_xr.rio.crs.to_epsg()
        if np.nanmean(im_xr['B2'])>1e3:
            im_xr = xr.where(im_xr==dataset_dict[dataset]['no_data_value'], np.nan, 
                             im_xr / dataset_dict[dataset]['image_scalar'])
        else:
            im_xr = xr.where(im_xr==dataset_dict[dataset]['no_data_value'], np.nan, im_xr)
        # add NDSI band
        im_xr['NDSI'] = ((im_xr[dataset_dict[dataset]['NDSI_bands'][0]] - im_xr[dataset_dict[dataset]['NDSI_bands'][1]]) 
                             / (im_xr[dataset_dict[dataset]['NDSI_bands'][0]] + im_xr[dataset_dict[dataset]['NDSI_bands'][1]]))
        im_xr.rio.write_crs('EPSG:'+str(crs), inplace=True)
                
        # -----Classify image
        # check if classified image already exists in file
        im_classified_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_classified.nc'
        if os.path.exists(im_classified_path + im_classified_fn):
            print('Classified image already exists in file, continuing...')
            im_classified = xr.open_dataset(im_classified_path + im_classified_fn)
            # remove no data values
            im_classified = xr.where(im_classified==-9999, np.nan, im_classified)
        else:  
            # classify image
            im_classified = f.classify_image(im_xr, clf, feature_cols, crop_to_AOI, AOI_UTM, DEM,
                                             dataset_dict, dataset, im_classified_fn, im_classified_path)
            if type(im_classified)==str: # skip if error in classification
                continue
        
        # -----Delineate snowline(s)
        # check if snowline already exists in file
        snowline_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_snowline.csv'
        # if os.path.exists(snowlines_path + snowline_fn):
        #     print('Snowline already exists in file, continuing...')
        #     continue # no need to load snowline if it already exists
        # else:
        plot_results = True
        # create directory for figures if it doesn't already exist
        if (not os.path.exists(figures_out_path)) & plot_results:
            os.mkdir(figures_out_path)
            print('Created directory for output figures: ' + figures_out_path)
        snowline_df = delineate_snowline(im_xr, im_classified, site_name, AOI_UTM, dataset_dict, dataset, 
                                         im_date, snowline_fn, snowlines_path, figures_out_path, plot_results)
        plt.show()
        print('Accumulation Area Ratio =  ' + str(snowline_df['AAR'][0]))
        print(' ')


## 3. Landsat 8/9 SR

In [ ]:
# -----Query GEE for imagery (and download to L_im_path if necessary)
dataset = 'Landsat'
im_list = f.query_gee_for_imagery(dataset_dict, dataset, AOI_UTM, date_start, date_end, month_start, month_end,
                                  cloud_cover_max, mask_clouds, L_im_path, im_download)

In [ ]:
# -----Load trained classifier and feature columns
clf_fn = base_path+'inputs-outputs/Landsat_classifier_all_sites.joblib'
clf = load(clf_fn)
feature_cols_fn = base_path+'inputs-outputs/Landsat_feature_columns.json'
feature_cols = json.load(open(feature_cols_fn))

# -----Loop through images
if type(im_list)==str: # check that images were found
    print('No images found to classify, quitting...')
else:
    
    for i in tqdm(range(0, len(im_list))):
        
        # -----Subset image using loop index
        im_xr = im_list[i]
        im_date = str(im_xr.time.data[0])[0:19]
        print(im_date)
        
        # -----Adjust image for image scalar and no data values
        # replace no data values with NaN and account for image scalar
        crs = im_xr.rio.crs.to_epsg()
        # add NDSI band
        im_xr['NDSI'] = ((im_xr[dataset_dict[dataset]['NDSI_bands'][0]] - im_xr[dataset_dict[dataset]['NDSI_bands'][1]]) 
                         / (im_xr[dataset_dict[dataset]['NDSI_bands'][0]] + im_xr[dataset_dict[dataset]['NDSI_bands'][1]]))
        im_xr.rio.write_crs('EPSG:'+str(crs), inplace=True)
                
        # -----Classify image
        # check if classified image already exists in file
        im_classified_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_classified.nc'
        if os.path.exists(im_classified_path + im_classified_fn):
            print('Classified image already exists in file, continuing...')
            im_classified = xr.open_dataset(im_classified_path + im_classified_fn)
            # remove no data values
            im_classified = xr.where(im_classified==-9999, np.nan, im_classified)
        else:  
            # classify image
            im_classified = f.classify_image(im_xr, clf, feature_cols, crop_to_AOI, AOI_UTM, DEM,
                                             dataset_dict, dataset, im_classified_fn, im_classified_path)
            if type(im_classified)==str: # skip if error in classification
                continue
        
        # -----Delineate snowline(s)
        # check if snowline already exists in file
        snowline_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_snowline.csv'
        if os.path.exists(snowlines_path + snowline_fn):
            print('Snowline already exists in file, continuing...')
            continue # no need to load snowline if it already exists
        else:
            plot_results = True
            # create directory for figures if it doesn't already exist
            if (not os.path.exists(figures_out_path)) & plot_results:
                os.mkdir(figures_out_path)
                print('Created directory for output figures: '+figures_out_path)
            snowline_df = f.delineate_image_snowline(im_xr, im_classified, site_name, AOI_UTM, dataset_dict, dataset, 
                                                     im_date, snowline_fn, snowlines_path, figures_out_path, plot_results)
            # plt.show()
            print('Accumulation Area Ratio =  ' + str(snowline_df['AAR'][0]))
        print(' ')

## 4. PlanetScope SR

In [ ]:
# -----Read surface reflectance image file names
os.chdir(PS_im_path)
im_fns = sorted(glob.glob('*SR*.tif'))

# ----Mask clouds and cloud shadows in all images
plot_results = False
if mask_clouds:
    print('Masking images using cloud bitmask...')
    for im_fn in tqdm(im_fns):
        f.planetscope_mask_image_pixels(PS_im_path, im_fn, PS_im_masked_path, save_outputs, plot_results)
# read masked image file names
os.chdir(PS_im_masked_path)
im_masked_fns = sorted(glob.glob('*_mask.tif'))
    
# -----Mosaic images captured within same hour
print('Mosaicking images captured in the same hour...')
if mask_clouds: 
    f.planetscope_mosaic_images_by_date(PS_im_masked_path, im_masked_fns, PS_im_mosaics_path, AOI_UTM)
    print(' ')
else:
    f.planetscope_mosaic_images_by_date(PS_im_path, im_fns, PS_im_mosaics_path, AOI_UTM)
    print(' ')
    
# -----Load trained classifier and feature columns
clf_fn = base_path+'inputs-outputs/PlanetScope_classifier_all_sites.joblib'
clf = load(clf_fn)
feature_cols_fn = base_path+'inputs-outputs/PlanetScope_feature_columns.json'
feature_cols = json.load(open(feature_cols_fn))
dataset = 'PlanetScope'

# -----Adjust image radiometry
# read mosaicked image file names
os.chdir(PS_im_mosaics_path)
im_mosaic_fns = sorted(glob.glob('*.tif'))
# create polygon(s) of the top and bottom 20th percentile elevations within the AOI
polygons_top, polygons_bottom = f.create_aoi_elev_polys(AOI_UTM, DEM)
# loop through images
for im_mosaic_fn in tqdm(im_mosaic_fns[104:]):
    
    # -----Open image mosaic
    im_da = xr.open_dataset(PS_im_mosaics_path + im_mosaic_fn)
    # determine image date from image mosaic file name
    im_date = im_mosaic_fn[0:4] + '-' + im_mosaic_fn[4:6] + '-' + im_mosaic_fn[6:8] + 'T' + im_mosaic_fn[9:11] + ':00:00'
    im_dt = np.datetime64(im_date)
    print(im_date)
    
    # -----Adjust radiometry
    im_adj, im_adj_method = f.planetscope_adjust_image_radiometry(im_da, im_dt, polygons_top, polygons_bottom, dataset_dict, skip_clipped)
    if type(im_adj)==str: # skip if there was an error in adjustment
        continue
    
    # -----Classify image
    im_classified_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_classified.nc'
    if os.path.exists(im_classified_path + im_classified_fn):
        print('Classified image already exists in file, loading...')
        im_classified = xr.open_dataset(im_classified_path + im_classified_fn)
        # remove no data values
        im_classified = xr.where(im_classified==-9999, np.nan, im_classified)
    else:
        im_classified = f.classify_image(im_adj, clf, feature_cols, crop_to_AOI, AOI_UTM, DEM,
                                         dataset_dict, dataset, im_classified_fn, im_classified_path)
    if type(im_classified)==str:
        continue    
    
    # -----Delineate snowline(s)
    plot_results=True
    # create directory for figures if it doesn't already exist
    if (not os.path.exists(figures_out_path)) & plot_results:
        os.mkdir(figures_out_path)
        print('Created directory for output figures: '+figures_out_path)
    # check if snowline already exists in file
    snowline_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_snowline.csv'
    if os.path.exists(snowlines_path + snowline_fn):
        print('Snowline already exists in file, skipping...')
    else:
        snowline_df = f.delineate_image_snowline(im_adj, im_classified, site_name, AOI_UTM, dataset_dict, dataset, 
                                                 im_date, snowline_fn, snowlines_path, figures_out_path, plot_results)
        # plt.show()
        print('Accumulation Area Ratio =  ' + str(snowline_df['AAR'][0]))
    print(' ')

## _Optional_: Compile individual figures into a single .gif file

In [ ]:
### Modify the strings below according to your file names ###

# identify the string that is present in all filenames of the figures that you want to compile
fig_fns_str = '*' + site_name + '_' + dataset + '_*snow-cover.png'
# define the output .gif filename
gif_fn = site_name + '_' + dataset + '_' + date_start.replace('-','') + '_' + date_end.replace('-','') + 'snow-cover.gif' 

# -----Make a .gif of output images
from PIL import Image as PIL_Image
from IPython.display import Image as IPy_Image
os.chdir(figures_out_path)
fig_fns = glob.glob(fig_fns_str) # load all output figure file names
fig_fns = sorted(fig_fns) # sort chronologically

# grab figures date range for .gif file name
frames = [PIL_Image.open(im) for im in fig_fns]
frame_one = frames[0]
frame_one.save(figures_out_path + gif_fn, format="GIF", append_images=frames, save_all=True, duration=2000, loop=0)
print('GIF saved to file:' + figures_out_path + gif_fn)


# -----Clean up: delete individual figure files
for fn in fig_fns:
    os.remove(os.path.join(figures_out_path, fn))
print('Individual figure files deleted.')

# -----Display .gif
IPy_Image(filename = figures_out_path + gif_fn)